In [1]:
# !pip install setuptools

In [2]:
# !pip install -r requirements.txt

In [1]:
# Tengo un dataset en data/train/ que son imágenes para entrenar un modelo de clasificación de redes neuronales convolucionales
# Quiero que el modelo sea capaz de clasificar imágenes de gatos y perros

# 1. Cargar las imágenes
# 2. Preprocesar las imágenes
# 3. Crear el modelo
# 4. Entrenar el modelo
# 5. Evaluar el modelo
# 6. Guardar el modelo
# 7. Hacer predicciones

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers, models, regularizers


In [2]:
print("GPUs disponibles:", len(tf.config.list_physical_devices('GPU')))

GPUs disponibles: 0


In [3]:
train_dir = "data/train"
test_dir = "data/test"

# Carga los datasets
train_dataset = image_dataset_from_directory(
    train_dir,
    labels='inferred',         # Las etiquetas se infieren del nombre de la carpeta
    label_mode='int',          # Etiquetas como enteros
    image_size=(128, 128),     # Redimensiona las imágenes a 128x128
    batch_size=32,             # Tamaño del lote
    shuffle=True               # Mezcla los datos
)

test_dataset = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    shuffle=False
)
# Inspección de los datos
class_names = train_dataset.class_names
print(f"Clases detectadas: {class_names}")

Found 4733 files belonging to 2 classes.
Found 1184 files belonging to 2 classes.
Clases detectadas: ['chihuahua', 'muffin']


In [4]:

# Opcional: Normalización de imágenes (valores entre 0 y 1)
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))



# Opcional: Configuración para prefetching (mejora el rendimiento durante el entrenamiento)
# AUTOTUNE = tf.data.AUTOTUNE
# train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
# test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)


In [7]:
# Definir el modelo de la CNN
model = models.Sequential([
    # Primera capa convolucional
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),

    # Segunda capa convolucional
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Tercera capa convolucional
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Aplanar y agregar capas densas
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),

    # Capa de salida (número de clases)
    layers.Dense(len(class_names), activation='softmax')  # len(class_names) = número de clases
])

# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
model.summary()

# Entrenar el modelo
model.fit(train_dataset, epochs=10)

# Evaluar el modelo
loss, accuracy = model.evaluate(test_dataset)
print(f"Pérdida: {loss}")
print(f"Exactitud: {accuracy}")


C:\Users\raval\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,313,026 (12.64 MB)

 Trainable params: 3,313,026 (12.64 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 52s 317ms/step - accuracy: 0.6673 - loss: 0.6002
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 48s 321ms/step - accuracy: 0.8378 - loss: 0.3713
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 52s 347ms/step - accuracy: 0.8871 - loss: 0.2740
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 48s 324ms/step - accuracy: 0.8987 - loss: 0.2394
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 49s 326ms/step - accuracy: 0.9262 - loss: 0.1869
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 49s 326ms/step - accuracy: 0.9423 - loss: 0.1418
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 48s 320ms/step - accuracy: 0.9656 - loss: 0.0895
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 49s 331ms/step - accuracy: 0.9781 - loss: 0.0542
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 54s 359ms/step - accuracy: 0.9809 - loss: 0.0502
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 90s 605ms/step - accuracy: 0.9880 - loss: 0.0299
37/37 ━━━━━━━━━━━━━━━━━━━━ 10s 229ms/step - accuracy: 0.8960 - loss: 0.4496
Pérdida: 0.4936676025390625
Exact

In [ ]:
model_comb1 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),  # Dropout en la primera capa convolucional

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')  # Número de clases
])

# Compilar el modelo
model_comb1.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
model_comb1.summary()

# Entrenar el modelo
model_comb1.fit(train_dataset, epochs=10)

# Evaluar el modelo
loss, accuracy = model_comb1.evaluate(test_dataset)
print(f"Pérdida: {loss}")
print(f"Exactitud: {accuracy}")


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 126, 126, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 61, 61, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 28, 28, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,314,690 (12.64 MB)

 Trainable params: 3,313,858 (12.64 MB)

 Non-trainable params: 832 (3.25 KB)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 185s 1s/step - accuracy: 0.7804 - loss: 0.5157
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.8635 - loss: 0.3371
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 155s 1s/step - accuracy: 0.8799 - loss: 0.2892
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 111s 746ms/step - accuracy: 0.9020 - loss: 0.2409
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 88s 592ms/step - accuracy: 0.9238 - loss: 0.2062
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 87s 583ms/step - accuracy: 0.9382 - loss: 0.1750
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 87s 586ms/step - accuracy: 0.9427 - loss: 0.1526
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 88s 589ms/step - accuracy: 0.9487 - loss: 0.1278
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 85s 571ms/step - accuracy: 0.9388 - loss: 0.1527
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 79s 534ms/step - accuracy: 0.9696 - loss: 0.0940
37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 133ms/step - accuracy: 0.5928 - loss: 1.0846
Pérdida: 0.696784496307373
Exactitud: 0

In [7]:
model_comb2 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    # Primera capa densa con regularización L2 y Dropout
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Dropout del 30%

    # Segunda capa densa con regularización L2 y Dropout
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Dropout del 30%

    # Tercera capa densa con regularización L2
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),

    # Capa de salida (número de clases)
    layers.Dense(len(class_names), activation='softmax')  # len(class_names) = número de clases
])

# Optimizer con decay
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
model_comb2.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model_comb2.summary()

# Entrenar el modelo
model_comb2.fit(train_dataset, epochs=10)

# Evaluar el modelo
loss, accuracy = model_comb2.evaluate(test_dataset)

print(f"Pérdida: {loss}")
print(f"Exactitud: {accuracy}")

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,558,210 (25.02 MB)

 Trainable params: 6,557,762 (25.02 MB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 61s 396ms/step - accuracy: 0.7049 - loss: 8.9175
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 57s 386ms/step - accuracy: 0.7862 - loss: 4.2664
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 57s 386ms/step - accuracy: 0.8587 - loss: 2.4857
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 392ms/step - accuracy: 0.8765 - loss: 1.8675
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 388ms/step - accuracy: 0.8883 - loss: 1.5809
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 387ms/step - accuracy: 0.9018 - loss: 1.2572
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 57s 386ms/step - accuracy: 0.9140 - loss: 1.0713
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 56s 378ms/step - accuracy: 0.9212 - loss: 0.9447
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 57s 381ms/step - accuracy: 0.9237 - loss: 0.8509
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 56s 379ms/step - accuracy: 0.9314 - loss: 0.7653
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step - accuracy: 0.8960 - loss: 0.7936
Pérdida: 1.2712395191192627
Exactit

In [5]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay
# Decaimiento de la tasa de aprendizaje
learning_rate_schedule = ExponentialDecay(
    initial_learning_rate=0.01,  # Tasa de aprendizaje inicial
    decay_steps=1000,           # Número de pasos para aplicar el decay
    decay_rate=0.9,             # Factor de decaimiento
    staircase=False             # Decaimiento continuo
)
model_comb3 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),  # Dropout en las capas convolucionales

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    # Primera capa densa con L2 y Dropout
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Apaga el 30% de las neuronas

    # Segunda capa densa con L2 y Dropout
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Apaga el 30% de las neuronas

    # Tercera capa densa con L2
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),

    # Capa de salida
    layers.Dense(len(class_names), activation='softmax')  # Número de clases
])

# Optimizer con decay
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule)
model_comb3.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model_comb3.summary()

# Entrenar el modelo
model_comb3.fit(train_dataset, epochs=10)

# Evaluar el modelo
loss, accuracy = model_comb3.evaluate(test_dataset)

print(f"Pérdida: {loss}")

print(f"Exactitud: {accuracy}")

c:\Magister\3er trimestre\MLA\MLA_T2\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,558,210 (25.02 MB)

 Trainable params: 6,557,762 (25.02 MB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 63s 406ms/step - accuracy: 0.5717 - loss: 44.3073
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 400ms/step - accuracy: 0.7516 - loss: 2.4271
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 402ms/step - accuracy: 0.7711 - loss: 1.9972
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 61s 412ms/step - accuracy: 0.7963 - loss: 1.5462
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 402ms/step - accuracy: 0.8111 - loss: 1.4821
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 59s 398ms/step - accuracy: 0.8216 - loss: 1.2005
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 404ms/step - accuracy: 0.8148 - loss: 1.1952
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 63s 427ms/step - accuracy: 0.8298 - loss: 1.1745
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 81s 419ms/step - accuracy: 0.8288 - loss: 1.1205
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 63s 423ms/step - accuracy: 0.8447 - loss: 1.1384
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - accuracy: 0.8772 - loss: 1.2057
Pérdida: 1.1980923414230347
Exacti